In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout
import matplotlib.pyplot as plt
import datetime as dt
import plotly.offline as pyo
import plotly.express as px
from pandas_datareader import data as pdr
import yfinance as yf

In [ ]:
dt.date.today()

In [ ]:
stocks = pd.read_csv('Output 2.csv')

In [ ]:
stocks_list = stocks['Ticker'].values.tolist()

In [ ]:
stocks_list

In [ ]:
end = dt.datetime.now()
start = end - dt.timedelta(days = 3650)
# start = '2010-01-01'
# end = '2019-12-31'
# start=dt.datetime(2017, 2, 9)
# end=dt.datetime(2017, 5, 24)

In [ ]:
yf.pdr_override()

In [ ]:
df = pdr.DataReader('AAPL', start ,end)
df.head()

In [ ]:
df = df.reset_index()
df.head()

In [ ]:
ma100 = df.Close.rolling(100).mean()
ma200 = df.Close.rolling(200).mean()
df.insert(3, "Moving average 100", ma100)
df.insert(4, "Moving average 200", ma200)

In [ ]:
df.tail()

In [ ]:
fig1 = px.line(df, y =['Close','Moving average 100'])
fig1.show()

In [ ]:
fig2 = px.line(df, x=df.Date , y =['Close','Moving average 200'])
fig2.show()

In [ ]:
df.shape

In [ ]:
#Splitting Data into Training and Testing

train_data = pd.DataFrame(df['Close'][0:int(0.8*len(df))])
test_data = pd.DataFrame(df['Close'][int(0.8*len(df)):int(len(df))])

print(train_data.shape)
print(test_data.shape)

In [ ]:
test_data.head()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_training_array = scaler.fit_transform(train_data)
data_training_array

In [ ]:
x_train= []
y_train= []

for i in range(100, train_data.shape[0]):
    x_train.append(data_training_array[i-100: i])
    y_train .append(data_training_array[i,0])
    
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
import os
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join("Logs_sales_prediciton_model")

In [ ]:
cb = TensorBoard(log_dir = log_dir)

In [ ]:
model = Sequential()

model.add(LSTM(units =64, return_sequences = True,input_shape = (x_train.shape[1],1)))
model.add(Dropout(0.5))

model.add(LSTM(units =128,  return_sequences = True,))
model.add(Dropout(0.6))

model.add(LSTM(units =264, return_sequences = True,))
model.add(Dropout(0.7)) 

model.add(LSTM(units =512))
model.add(Dropout(0.9))


model.add(Dense(units = 1))

In [ ]:
# Add 2 dense hidden layers
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))

In [ ]:
#model.add(Dense(output_dim, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs = 5, callbacks = [cb])

In [ ]:
past_100_days = train_data.tail(1000)

In [ ]:
final_df = past_100_days.append(test_data, ignore_index = True)

In [ ]:
input_data = scaler.fit_transform(final_df)

In [ ]:
input_data.shape

In [ ]:
x_test = []
y_test = []

for i in range(100 , input_data.shape[0]):
    x_test.append(input_data[i-100 : i])
    y_test.append(input_data[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)
print(x_test.shape)
print(y_test.shape)

# Making Prediction

In [ ]:
y_predicted = model.predict(x_test)

In [ ]:
y_predicted.shape

In [ ]:
scaler.scale_

In [ ]:
scale_factor = 1/0.00761325
y_predicted =  y_predicted * scale_factor
y_test = y_test * scale_factor

In [ ]:
y_predicted.shape

In [ ]:
df.head()

In [ ]:
y_predicted = y_predicted.reshape(-1)

In [ ]:
temp_data = list(zip(df.Date[y_test.shape[0] : df.shape[0]],y_test, y_predicted))
pre = pd.DataFrame(temp_data,columns =['Date','Testing Data','Prediction'])

In [ ]:
pre

In [ ]:
fig4 = px.line(pre, x=pre.Date, y=['Testing Data','Prediction'])
fig4.show()